<a href="https://colab.research.google.com/github/3dsf/ArtLine/blob/main/videoProcessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Processing with AI models
based on the ffmpeg-python tensorflow implementation 
and using the models of 
https://github.com/vijishmadhavan



# Install Libs

In [1]:
!pip install youtube-dl fastai==1.0.61 ffmpeg-python

# Select
## -- Model
## -- Video
## -- Output Name
*and collect metadata*
 

In [19]:
import os

modelToRun = "ArtLine_1024.pkl" #@param ["ArtLine_500.pkl", "ArtLine_650.pkl", "ArtLine_1024.pkl", "SkinDeep.pkl", "SkinDeep_1280.pkl"]
pathToModel = os.path.join("/content/drive/",modelToRun)

downloadModel = {
    "ArtLine_500.pkl": "https://www.dropbox.com/s/p9lynpwygjmeed2/ArtLine_500.pkl",
    "ArtLine_650.pkl": "https://www.dropbox.com/s/starqc9qd2e1lg1/ArtLine_650.pkl",
    "ArtLine_1024.pkl": "https://www.dropbox.com/s/rq90q9lr9arwdp8/ArtLine_1024%20%281%29.pkl",
    "SkinDeep.pkl": "https://www.dropbox.com/s/5mmcqao4mozpube/SkinDeep.pkl?dl=1",
    "SkinDeep_1280.pkl": "https://www.dropbox.com/s/wxty56nhidusojr/SkinDeep_1280.pkl"
}
if os.path.isfile(pathToModel) == False : 
    if os.path.isfile(modelToRun) == False :
        print("Downloading Model")
        download = downloadModel[modelToRun]
        !wget -O $modelToRun $download
        pathToModel = modelToRun
    else :
        print("Found Local Version")
        pathToModel = modelToRun

videoURL = "https://www.youtube.com/watch?v=V_aoPGUqtV8" #@param {type:"string"}
!rm input.mp4  #required
!time(youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=acc]/mp4' --output "input.%(ext)s"  $videoURL)
output_name = "merk.artLine_1024.mp4" #@param {type:"string"}

import subprocess
AUDIO = False
process = subprocess.Popen(['ffmpeg', '-hide_banner', '-i', 'input.mp4', '-y' ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True)
for line in process.stdout:
    print(line)
    if ' Video:' in line:
        l_split = line.split(',')
        #print('---------printing line ", line)
        for segment in l_split[1:]:
            if 'fps' in segment:
                    s = segment.strip().split(' ')
                    fps = float(s[0])
            if 'x' in segment:
                    s = segment.strip().split('x')
                    width = int(s[0])
                    s2 = s[1].split(' ')
                    height = int(s2[0])
    if 'Duration:' in line:
        s = line.split(',')
        ss = s[0].split(' ')
        sss = ss[3].strip().split(':')
        seconds = float(sss[0])*60*60 + float(sss[1])*60 + float(sss[2])
    if 'Audio:' in line:
        AUDIO = True

print('fps = ', str(fps))
print('width = ', str(width))
print('height = ', str(height))
print('seconds = ', str(seconds))
print('AUDIO = ', AUDIO)

Found Local Version
[youtube] V_aoPGUqtV8: Downloading webpage
[download] Destination: input.mp4
[download] 100% of 11.51MiB in 00:00

real	0m1.293s
user	0m0.645s
sys	0m0.069s
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':

  Metadata:

    major_brand     : mp42

    minor_version   : 0

    compatible_brands: isommp42

    creation_time   : 2019-11-28T10:33:04.000000Z

  Duration: 00:04:12.77, start: 0.000000, bitrate: 382 kb/s

    Stream #0:0(und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p(tv, bt709), 640x360 [SAR 1:1 DAR 16:9], 283 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)

    Metadata:

      creation_time   : 2019-11-28T10:33:04.000000Z

      handler_name    : ISO Media file produced by Google Inc. Created on: 11/28/2019.

    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 95 kb/s (default)

    Metadata:

      creation_time   : 2019-11-28T10:33:04.000000Z

      handler_name    : ISO Media file produc

# Process Video

In [20]:
import os
import logging as logger

from torchvision import transforms as T
from fastai.utils.mem import *
from fastai.vision import open_image, load_learner, Image, torch, pil2tensor, image2np

import ffmpeg, cv2
import numpy as np

#progress bar
from IPython.display import HTML, display
from tqdm import *

#There is scaling warning that might come up, and this block supresses user warnings
#Comment out this block if your don't mind seeing the warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  

### Progress bar
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

### Class required for model
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]

    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)

    def __del__(self): self.hooks.remove()

### DETERMINE IF CUDA AVAILABLE and LOAD MODEL
def modelDeviceLoadSelect():  
    if torch.cuda.is_available():
        def load_model():
            global USEgPU
            learn = load_learner('.', pathToModel, device=0 )
            USEgPU = True
            print("INFERENCE DEVICE : cuda")
            return learn
    else:
        def load_model():
            learn = load_learner('.', pathToModel, device='cpu')
            print("INFERENCE DEVICE : cpu")
            return learn
    learn=load_model()
    return learn

### Functions based on ffmpeg-python video tensorflow example
def readFrameAsNp(ffmpegDecode, width, height):
    logger.debug('Reading frame')

    # Note: RGB24 == 3 bytes per pixel.
    frame_size = width * height * 3
    in_bytes = ffmpegDecode.stdout.read(frame_size)
    if len(in_bytes) == 0:
        frame = None
    else:
        assert len(in_bytes) == frame_size
        frame = (
            np
            .frombuffer(in_bytes, np.uint8)
            .reshape([height, width, 3])
        )
    return frame

def writeFrameAsByte(ffmpegEncode, frame):
    logger.debug('Writing frame')
    ffmpegEncode.stdin.write(
        frame
        .astype(np.uint8)
        .tobytes()
    )

def vid2np(in_filename):
    logger.info('vid2np() -- Decoding to pipe')
    codec = 'h264'
    args = (
            ffmpeg
            .input(in_filename,
                **{'c:v': codec})
            .output('pipe:', format='rawvideo', pix_fmt='rgb24')
            .global_args("-hide_banner")
            .compile()
    )
    return subprocess.Popen(args, stdout=subprocess.PIPE)

def np2vid(out_filename, fps_out, in_file, widthOut, heightOut):
    logger.info('np2vid() encoding from pipe')
    global AUDIO
    codec = 'h264'
    if AUDIO == True :
        pipeline2 = ffmpeg.input(in_file)
        audio = pipeline2.audio
        args = (
            ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24',
                s='{}x{}'.format(widthOut, heightOut),
                framerate=fps_out )
            .output(audio, out_filename , pix_fmt='yuv420p', **{'c:v': codec}, 
                shortest=None, acodec='copy')
            .global_args("-hide_banner")
            .overwrite_output()
            .compile()
        )
    else:
        args = (
            ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', 
                s='{}x{}'.format(widthOut, heightOut), 
                framerate=fps_out )
            .output(out_filename , pix_fmt='yuv420p', **{'c:v': codec})
            .global_args("-hide_banner")
            .overwrite_output()
            .compile()
        )
    return subprocess.Popen(args, stdin=subprocess.PIPE)

### The model changes the resolution, processes blank to find new resolution
def getOutputResolution():
    #process a blank frame and return dimesions
    blank = np.zeros([height,width,3],dtype=np.uint8)
    blank.fill(255)
    fastAI_image = Image(pil2tensor(blank, dtype=np.float32).div_(255))
    p,img_hr,b = learn.predict(fastAI_image)
    im = image2np(img_hr)
    x = im.shape
    out_height = x[0]
    out_width = x[1]
    return int(out_width), int(out_height)

### This is where all the magic happens
def processFrame(frame) :
    global INCR
    ### Frame comes in as np array
    #Load image in fastai's framework as an image
    fastAI_image = Image(pil2tensor(frame, dtype=np.float32).div_(255))
    # Inference
    p,img_hr,b = learn.predict(fastAI_image)
    # Convert output tensor into np array
    im = image2np(img_hr)
    # alpha and beta control line output darkness / warmness
    norm_image = cv2.normalize(im, None, alpha=-60, beta=260, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    INCR += 1
    # enabling the next 2 lines will also output images when processing videos
    #outCV2 = cv2.cvtColor(norm_image, cv2.COLOR_RGB2BGR )
    #cv2.imwrite(output_name+ str(INCR) + ".png", outCV2)  # INCR is just a frame counter
    return norm_image 

if __name__ == '__main__':

    INCR = 0

    learn = modelDeviceLoadSelect()
    outWidth, outHeight = getOutputResolution()
    estimatedFrames = fps * seconds

    print('Model = ', pathToModel)
    print('*** Video In***')
    print('fps = ', str(fps))
    print('width = ', str(width))
    print('height = ', str(height))
    print('seconds = ', str(seconds))
    print('AUDIO = ', AUDIO)
    print()
    print('*** Video Out***')
    print('outWidth = ', str(outWidth))
    print('outHeight = ', str(outHeight))
    print('output_name = ', output_name)
    print()
    #progress bar
    print('estimatedFrames = ', estimatedFrames)
    out = display(progress(0, 100), display_id=True)

    inputVid = 'input.mp4'
    ffmpegDecode = vid2np(inputVid)
    ffmpegEncode = np2vid(output_name, fps, inputVid, outWidth, outHeight)

    while True:
        timeMark = time.process_time()
        in_frame = readFrameAsNp(ffmpegDecode, width, height)
        if in_frame is None:
            logger.info('End of input stream')
            break

        logger.debug('Processing frame')
        out_frame = processFrame(in_frame)
        writeFrameAsByte(ffmpegEncode, out_frame)

        #progress bar
        out.update(progress(INCR, estimatedFrames))
        minutesRemaining = str(round((estimatedFrames-INCR)*(time.process_time()-timeMark)/60))
        print("\rEstimated Minutes Remaining = ", minutesRemaining, end="")

    logger.info('Waiting for ffmpegDecode')
    ffmpegDecode.wait()

    logger.info('Waiting for ffmpegEncode')
    ffmpegEncode.stdin.close()
    ffmpegEncode.wait()

    logger.info('Done')

INFERENCE DEVICE : cuda
Model =  SkinDeep_1280.pkl
*** Video In***
fps =  23.98
width =  640
height =  360
seconds =  252.77
AUDIO =  True

*** Video Out***
outWidth =  1280
outHeight =  720
output_name =  merk.sd1280.mp4

estimatedFrames =  6061.4246


Estimated Minutes Remaining =  0

# Download result

In [21]:
from google.colab import files

files.download(output_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>